In [11]:
import math
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import networkx as nx
import matplotlib.pyplot as plt
import operator

In [12]:
def entropy(attr_name):
    entropy=0
    row_del_dict = dict()
    inner_row_del_dict = dict()
    #getting column contents of current attribute name from data extracted from csv file
    attr_list=list(data[attr_name])
    #print(attr_list)
    if attr_name == 'fast':
        class_attr_list=set(attr_list)#get number of unique classes in attribute
        # print(class_attr_list)
        all_classes=list(class_attr_list)#convert back to list
        # print(all_classes)
        for x in all_classes:
            probability=attr_list.count(x)/len(attr_list)
            entropy-=(probability*math.log2(probability))
            #print(len(attr_list))
    else:
        merged_class=list(zip(attr_list,list(data['fast'])))
        #entropy=0
        class_attr_list=set(merged_class)#get number of unique classes in attribute
        # print(class_attr_list) 
        all_classes=list(class_attr_list)#convert back to list
        # print(all_classes) --> unique combinations --> [('large', 'no'), ('large', 'yes'), ('small', 'yes'), ('small', 'no'), ('medium', 'no'), ('medium', 'yes')]
        for x in all_classes:       
                # print(x)    ---> represents child combination e.g  ('large', 'no'), x[0] is 'large' or 'small',or 'medium'  for engine
                probability=merged_class.count(x)/attr_list.count(x[0])
                # print(attr_list.count(x[0]),len(attr_list),probability,math.log2(probability))
                #entropy of child combinations
                sub_entropy = probability*math.log2(probability)
                #dictionary to determine rows to be deleted in table reduction
                inner_row_del_dict[x[0]]=sub_entropy #key is either large,medium,small and value is its entropy for engine for example
                row_del_dict[attr_name]=inner_row_del_dict
                entropy-=(attr_list.count(x[0])/len(attr_list))*(sub_entropy)
        # print(inner_row_del_dict) 
        # print(row_del_dict)
    #print("Target Attribute Entropy: ",entropy, "\n")
    return entropy,row_del_dict

In [13]:
def information_gain(target_entopy,attr_entopy):
        info_gain = target_entopy-attr_entopy
        return info_gain

In [14]:
def root_node_election(target_entopy):
    rst_dict=dict()
    for x in data_headers[:-1]:#calculate entopies of all minus the target
        #print("Entropy of ",x," is ",entropy(x))
        attr_entopy,attr_row_del_dict = entropy(x)
        info_gain = information_gain(target_entopy,attr_entopy)
        print("IG of ",x," is ",info_gain)
        #store results in a dictionary
        rst_dict[x]=info_gain
        print(attr_row_del_dict)
    # print(rst_dict)
    return max(rst_dict.items(),key=operator.itemgetter(1))[0],attr_row_del_dict

In [15]:
def reduce_training_dataset(data,delete_rows,selected_root_node):
    for item in delete_rows:
        type(data)
#       data = data[data.eval(selected_root_node) != item]
#     print(data)
#     data = data.drop(selected_root_node,axis=1)
#     print(data.head(0))
    #data_headers.remove(selected_root_node) #delete header of choosen root node
    return data

In [16]:
data_headers=['engine','turbo','weight','fueleco','fast']

In [4]:
# from google.colab import files
# upload = files.upload()

In [17]:
data = pd.read_csv('https://pastebin.com/raw/wKhPrSXi',names=data_headers,header=None ) # Using a url instead of file upload
# data = pd.read_csv('id3_data.csv',names=data_headers,header=None )

In [18]:
data

,engine,turbo,weight,fueleco,fast
0,small,no,average,good,no
1,small,no,light,average,no
2,small,yes,average,bad,yes
3,medium,no,heavy,bad,yes
4,large,no,average,bad,yes
5,medium,no,light,bad,no
6,large,yes,heavy,bad,no
7,large,no,heavy,bad,no
8,medium,yes,light,bad,yes
9,large,no,average,bad,yes


In [19]:
target_entopy,attr_row_del_dict = entropy('fast')
attr_row_del_dict

{}

In [20]:
target_entopy

0.9182958340544896

In [21]:
print("Entropy of fast is: ",target_entopy)
selected_root_node,delete_rows = root_node_election(target_entopy)

Entropy of fast is:  0.9182958340544896
IG of  engine  is  0.06797000057692504
{'engine': {'medium': -0.44217935649972373, 'small': -0.21919533819482817, 'large': -0.5}}
IG of  turbo  is  0.17034377079624807
{'turbo': {'no': -0.2575424759098898, 'yes': -0.5, 'no ': 0.0}}
IG of  weight  is  0.061312302569600075
{'weight': {'heavy': -0.46438561897747244, 'average': -0.5, 'light': -0.31127812445913283}}
IG of  fueleco  is  0.2516291673878229
{'fueleco': {'bad': -0.5, 'good': 0.0, 'average': 0.0}}


In [22]:
selected_root_node 

'fueleco'

In [23]:
delete_rows

{'fueleco': {'bad': -0.5, 'good': 0.0, 'average': 0.0}}

In [24]:
print("Smaller entopies of root node are: ",delete_rows)
delete_row=[]

Smaller entopies of root node are:  {'fueleco': {'bad': -0.5, 'good': 0.0, 'average': 0.0}}


In [25]:
for row,value in delete_rows[selected_root_node].items():
    if value==1 or value==0:#arrest values with extreem entopies
        delete_row.append(row)#store them in a list

In [26]:
delete_row

['good', 'average']

In [27]:
print("The selected root node is: ",selected_root_node)   
print("The rows to be deleted are: ",delete_row)

The selected root node is:  fueleco
The rows to be deleted are:  ['good', 'average']


In [28]:
data = reduce_training_dataset(data,delete_row,selected_root_node)

ValueError: unknown type object

In [0]:
decision_tree_dict = dict()
decision_tree_dict[selected_root_node]=delete_row
decision_tree_dict  

In [0]:
myList= []
for item in delete_row:
     data1 = data.loc[data[selected_root_node] != item]
data1